In [43]:
import scipy.io as sio   
import argparse
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import r2_score
from scipy.stats.stats import pearsonr
import pickle
from sklearn.linear_model import LinearRegression
import xgboost
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVR
import h5py
import pickle

In [44]:
x_train= {}
x_test = {}
y_train = {}
y_test= {}
location_train= {}
location_test= {}
feature_all_train = {}
feature_all_test= {}
aoa_train={}
aoa_test={}
toa_train={}
toa_test={}
rss_train={}
rss_test={}
f = h5py.File('data_cross_250')
for i in range(4):
    x_train[i] = f[str(i)+'/loc_x_train'][:]
    x_test[i] = f[str(i)+'/loc_x_test'][:]
    y_train[i] = f[str(i)+'/loc_y_train'][:] 
    y_test[i] = f[str(i)+'/loc_y_test'][:]
    location_train[i] = f[str(i)+'/loc_train'][:,:]
    location_test[i] = f[str(i)+'/loc_test'][:,:]
    feature_all_train[i] =f[str(i)+'/feature_train'][:,:]
    feature_all_test[i] = f[str(i)+'/feature_test'][:,:]
    aoa_train[i] =f[str(i)+'/aoa_train'][:,:]
    aoa_test[i] = f[str(i)+'/aoa_test'][:,:]
    toa_train[i] =f[str(i)+'/toa_train'][:,:]
    toa_test[i] = f[str(i)+'/toa_test'][:,:]
    rss_train[i] = f[str(i)+'/toa_train'][:,:]
    rss_test[i] = f[str(i)+'/toa_test'][:,:]
aoa_toa_train={}
aoa_toa_test ={}
aoa_toa_rss_train={}
aoa_toa_rss_test ={}
for i in range(4):
    aoa_toa_train[i] = feature_all_train[i][:,6:18]
    aoa_toa_test[i] = feature_all_test[i][:,6:18]
    aoa_toa_rss_train[i] = feature_all_train[i][:,6:18]
    aoa_toa_rss_test[i] = feature_all_test[i][:,6:18]

In [35]:

average_dis_error = {}
for k in range(10):
    average_dis_error[k] = {}
    for m in range(10):
        kernel = 1.0 * DotProduct(sigma_0=k)**m
        reg = GaussianProcessRegressor(kernel = kernel, optimizer=None)
        regpre_x ={}
        regpre_y = {}
        r2_x = {}
        r2_y = {}
        pcc_x = {}
        pcc_y= {}
        dis_error = {}
        for i in range(4):   
            reg_x = reg.fit(feature_all_train[i][:,7:18],x_train[i])
            regpre_x[i] = reg_x.predict(feature_all_test[i][:,7:18])
            reg_y = reg.fit(feature_all_train[i][:,7:18],y_train[i])
            regpre_y[i] = reg_y.predict(feature_all_test[i][:,7:18])

     
            r2_x[i] = r2_score(regpre_x[i],x_test[i])
            pcc_x[i] = pearsonr(regpre_x[i],x_test[i])
            r2_y[i] = r2_score(regpre_y[i],y_test[i])
            pcc_y[i] = pearsonr(regpre_y[i],y_test[i])
    

            dis_x = {}
            dis_y = {}
            dis = {}
            for j in range(50):
                dis_y[j] = regpre_y[i][j] - y_test[i][j]
                dis_x[j] = regpre_x[i][j] - x_test[i][j]
                dis[j] = (dis_x[j]**2 + dis_y[j]**2)**(0.5)
            sum = np.array([])
            for j in range(50):
                sum = np.concatenate((sum,dis[j]))
            dis_error[i] = np.mean(abs(sum))
        sum = 0 
        for i in range(4):
            sum = sum+dis_error[i]
        average_dis_error[k][m] = sum/4.0

LinAlgError: ("The kernel, 1**2 * DotProduct(sigma_0=1) ** 9, is not returning a positive definite matrix. Try gradually increasing the 'alpha' parameter of your GaussianProcessRegressor estimator.", '131-th leading minor not positive definite')

In [34]:
average_dis_error

{0: {0: 506.524844128609}}

In [69]:
average_dis_error = {}
for k in range(1):
    average_dis_error[k] = {}
    for m in range(300,301):
        kernel = (m/10.0)*RBF(length_scale=2.8, length_scale_bounds=(10.0**(-m), 10.0**m))
        reg = GaussianProcessRegressor(kernel = kernel,alpha = 10**(-4.4), optimizer=None)
        regpre_x ={}
        regpre_y = {}
        r2_x = {}
        r2_y = {}
        pcc_x = {}
        pcc_y= {}
        dis_error = {}
        for i in range(4):   
            reg_x = reg.fit(feature_all_train[i][:,7:18],x_train[i])
            regpre_x[i] = reg_x.predict(feature_all_test[i][:,7:18])
            reg_y = reg.fit(feature_all_train[i][:,7:18],y_train[i])
            regpre_y[i] = reg_y.predict(feature_all_test[i][:,7:18])

     
            r2_x[i] = r2_score(regpre_x[i],x_test[i])
            pcc_x[i] = pearsonr(regpre_x[i],x_test[i])
            r2_y[i] = r2_score(regpre_y[i],y_test[i])
            pcc_y[i] = pearsonr(regpre_y[i],y_test[i])
    

            dis_x = {}
            dis_y = {}
            dis = {}
            for j in range(50):
                dis_y[j] = regpre_y[i][j] - y_test[i][j]
                dis_x[j] = regpre_x[i][j] - x_test[i][j]
                dis[j] = (dis_x[j]**2 + dis_y[j]**2)**(0.5)
            sum = np.array([])
            for j in range(50):
                sum = np.concatenate((sum,dis[j]))
            dis_error[i] = np.mean(abs(sum))
        sum = 0 
        for i in range(4):
            sum = sum+dis_error[i]
        average_dis_error[k][m] = sum/4.0

In [70]:
kr_error_table = pd.DataFrame.from_dict({(i): average_dis_error[i]
                           for i in average_dis_error.keys() 
                          },
                       orient='index')

In [71]:
np.asarray(kr_error_table)

array([[ 140.31498605]])

In [72]:
kr_error_table

,300
0,140.314986


In [7]:
kernel = RBF(length_scale=2.8, length_scale_bounds=(1e-05, 100000.0))
reg = GaussianProcessRegressor(kernel = kernel,alpha = 10**(-4.4), optimizer=None)
regpre_x ={}
regpre_y = {}
r2_x = {}
r2_y = {}
pcc_x = {}
pcc_y= {}
dis_error = {}
for i in range(4):   
    reg_x = reg.fit(feature_all_train[i][:,7:18],x_train[i])
    regpre_x[i] = reg_x.predict(feature_all_test[i][:,7:18])
    reg_y = reg.fit(feature_all_train[i][:,7:18],y_train[i])
    regpre_y[i] = reg_y.predict(feature_all_test[i][:,7:18])

     
    r2_x[i] = r2_score(regpre_x[i],x_test[i])
    pcc_x[i] = pearsonr(regpre_x[i],x_test[i])
    r2_y[i] = r2_score(regpre_y[i],y_test[i])
    pcc_y[i] = pearsonr(regpre_y[i],y_test[i])
    

    dis_x = {}
    dis_y = {}
    dis = {}
    for j in range(50):
        dis_y[j] = regpre_y[i][j] - y_test[i][j]
        dis_x[j] = regpre_x[i][j] - x_test[i][j]
        dis[j] = (dis_x[j]**2 + dis_y[j]**2)**(0.5)
    sum = np.array([])
    for j in range(50):
        sum = np.concatenate((sum,dis[j]))
    dis_error[i] = np.mean(abs(sum))
sum = 0 
for i in range(4):
    sum = sum+dis_error[i]
average_dis_error = sum/4.0
average_dis_error

171.5718193409021

In [ ]:
length_scale=2.8  alpha = 10**(-4.4)  1*RBF   length_scale_bounds=(1e-05, 100000.0)

In [80]:
import GPy

ImportError: cannot import name transformations